In [ ]:
# subscriptions/models.py
from django.db import models
from django.contrib.auth import get_user_model
from django.utils.crypto import get_random_string
import uuid

User = get_user_model()

class SubscriptionPlan(models.Model):
    TYPE_CHOICES = (
        ('personal', 'Personal'),
        ('business', 'Business'),
    )
    TIER_CHOICES = (
        ('free', 'Free'),
        ('pro', 'Pro'),
        ('max', 'Max'),
        ('team', 'Team'),
        ('enterprise', 'Enterprise'),
    )
    
    name = models.CharField(max_length=100)
    description = models.TextField()
    price = models.DecimalField(max_digits=10, decimal_places=2)
    type = models.CharField(max_length=20, choices=TYPE_CHOICES)
    tier = models.CharField(max_length=20, choices=TIER_CHOICES)
    credits_per_month = models.PositiveIntegerField(default=0)
    max_users = models.PositiveIntegerField(default=1)  # For business plans
    features = models.JSONField(default=dict)
    is_active = models.BooleanField(default=True)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        indexes = [
            models.Index(fields=['type', 'tier']),
            models.Index(fields=['is_active']),
        ]
    
    def __str__(self):
        return f"{self.type} - {self.tier}"

class UserProfile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE, related_name='profile')
    credits = models.PositiveIntegerField(default=0)
    referral_code = models.CharField(max_length=8, unique=True, editable=False)
    referred_by = models.ForeignKey(
        'self', on_delete=models.SET_NULL, null=True, blank=True, related_name='referrals'
    )
    streak_days = models.PositiveIntegerField(default=0)
    last_activity_date = models.DateField(null=True, blank=True)
    
    def save(self, *args, **kwargs):
        if not self.referral_code:
            self.referral_code = self._generate_unique_code()
        super().save(*args, **kwargs)
    
    @staticmethod
    def _generate_unique_code():
        # Generate a unique 8-character alphanumeric code
        return get_random_string(8).upper()
    
    class Meta:
        indexes = [
            models.Index(fields=['referral_code']),
            models.Index(fields=['credits']),  # For leaderboard queries
        ]
    
    def __str__(self):
        return f"Profile for {self.user.email}"

class Subscription(models.Model):
    STATUS_CHOICES = (
        ('active', 'Active'),
        ('canceled', 'Canceled'),
        ('expired', 'Expired'),
        ('trial', 'Trial'),
    )
    
    user = models.ForeignKey(User, on_delete=models.CASCADE, related_name='subscriptions')
    plan = models.ForeignKey(SubscriptionPlan, on_delete=models.PROTECT)
    status = models.CharField(max_length=20, choices=STATUS_CHOICES, default='trial')
    start_date = models.DateTimeField(auto_now_add=True)
    end_date = models.DateTimeField(null=True, blank=True)
    is_current = models.BooleanField(default=True)  # To easily query the current subscription
    stripe_subscription_id = models.CharField(max_length=100, blank=True, null=True)
    
    class Meta:
        indexes = [
            models.Index(fields=['user', 'is_current']),
            models.Index(fields=['status']),
            models.Index(fields=['end_date']),
        ]
    
    def __str__(self):
        return f"{self.user.email} - {self.plan.name} ({self.status})"

class CreditTransaction(models.Model):
    TYPE_CHOICES = (
        ('earn', 'Earn'),
        ('spend', 'Spend'),
        ('bonus', 'Bonus'),
        ('refund', 'Refund'),
        ('subscription', 'Subscription'),
    )
    
    user = models.ForeignKey(User, on_delete=models.CASCADE, related_name='credit_transactions')
    amount = models.IntegerField()  # Positive for earn, negative for spend
    transaction_type = models.CharField(max_length=20, choices=TYPE_CHOICES)
    description = models.CharField(max_length=255)
    reference_id = models.UUIDField(default=uuid.uuid4, editable=False)
    created_at = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        indexes = [
            models.Index(fields=['user', 'created_at']),
            models.Index(fields=['transaction_type']),
            models.Index(fields=['reference_id']),
        ]
    
    def __str__(self):
        return f"{self.user.email} - {self.transaction_type} {self.amount} credits"

In [ ]:
# subscriptions/serializers.py
from rest_framework import serializers
from django.contrib.auth import get_user_model
from .models import SubscriptionPlan, UserProfile, Subscription, CreditTransaction
from django_bleach.serializers import BleachModelSerializer
from django.core.validators import MinValueValidator, MaxValueValidator
import re

User = get_user_model()

class SubscriptionPlanSerializer(BleachModelSerializer):
    class Meta:
        model = SubscriptionPlan
        fields = ['id', 'name', 'description', 'price', 'type', 'tier', 
                  'credits_per_month', 'max_users', 'features']
        
class UserProfileSerializer(BleachModelSerializer):
    email = serializers.EmailField(source='user.email', read_only=True)
    username = serializers.CharField(source='user.username', read_only=True)
    
    class Meta:
        model = UserProfile
        fields = ['id', 'email', 'username', 'credits', 'referral_code', 'streak_days']
        read_only_fields = ['credits', 'referral_code', 'streak_days']

class SubscriptionSerializer(BleachModelSerializer):
    plan_details = SubscriptionPlanSerializer(source='plan', read_only=True)
    
    class Meta:
        model = Subscription
        fields = ['id', 'plan', 'plan_details', 'status', 'start_date', 'end_date']
        read_only_fields = ['status', 'start_date', 'end_date']

class CreditTransactionSerializer(BleachModelSerializer):
    class Meta:
        model = CreditTransaction
        fields = ['id', 'amount', 'transaction_type', 'description', 'reference_id', 'created_at']
        read_only_fields = ['id', 'reference_id', 'created_at']

class ReferralSerializer(serializers.Serializer):
    referral_code = serializers.CharField(
        required=True, 
        max_length=8,
        min_length=8,
        validators=[
            # Ensure only alphanumeric characters
            serializers.RegexValidator(
                regex=r'^[A-Z0-9]+$',
                message='Referral code must contain only uppercase letters and numbers'
            )
        ]
    )
    
    def validate_referral_code(self, value):
        # Check if the referral code exists
        try:
            profile = UserProfile.objects.get(referral_code=value)
            # Make sure the user is not referring themselves
            if self.context['request'].user.profile.referral_code == value:
                raise serializers.ValidationError("You cannot use your own referral code")
            return value
        except UserProfile.DoesNotExist:
            raise serializers.ValidationError("Invalid referral code")

class CreditUsageSerializer(serializers.Serializer):
    amount = serializers.IntegerField(
        required=True,
        validators=[
            MinValueValidator(1, message="Credit amount must be positive"),
            MaxValueValidator(1000, message="Cannot use more than 1000 credits at once")
        ]
    )
    purpose = serializers.CharField(required=True, max_length=255)
    
    def validate_purpose(self, value):
        # Sanitize and validate the purpose field
        if re.search(r'<script|javascript:|on\w+=', value, re.IGNORECASE):
            raise serializers.ValidationError("Purpose contains potentially unsafe content")
        return value

In [ ]:
# subscriptions/permissions.py
from rest_framework import permissions

class IsSubscriptionOwner(permissions.BasePermission):
    """
    Custom permission to only allow owners of a subscription to view it.
    """
    def has_object_permission(self, request, view, obj):
        # Check if user is the owner of the subscription
        return obj.user == request.user

In [ ]:
# subscriptions/views.py
from rest_framework import viewsets, status, permissions, generics
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.views import APIView
from django.db.models import Sum, F
from django.utils import timezone
from django.db import transaction
from django_ratelimit.decorators import ratelimit
from django_ratelimit.mixins import RatelimitMixin
from .models import SubscriptionPlan, UserProfile, Subscription, CreditTransaction
from .serializers import (
    SubscriptionPlanSerializer, UserProfileSerializer, 
    SubscriptionSerializer, CreditTransactionSerializer,
    ReferralSerializer, CreditUsageSerializer
)
from .permissions import IsSubscriptionOwner
import datetime

class SubscriptionViewSet(RatelimitMixin, viewsets.ModelViewSet):
    serializer_class = SubscriptionSerializer
    permission_classes = [permissions.IsAuthenticated, IsSubscriptionOwner]
    ratelimit_key = 'user'
    ratelimit_rate = '30/minute'
    ratelimit_method = 'GET'
    ratelimit_block = True
    
    def get_queryset(self):
        return Subscription.objects.filter(user=self.request.user).select_related('plan')
    
    def retrieve(self, request, *args, **kwargs):
        """Get user's current subscription details"""
        try:
            subscription = Subscription.objects.filter(
                user=request.user, 
                is_current=True
            ).select_related('plan').first()
            
            if not subscription:
                # Return the free plan details if no subscription
                free_plan = SubscriptionPlan.objects.filter(tier='free', is_active=True).first()
                # Create a free subscription if none exists
                if free_plan:
                    subscription = Subscription.objects.create(
                        user=request.user,
                        plan=free_plan,
                        status='active',
                        is_current=True,
                        end_date=timezone.now() + datetime.timedelta(days=365*10)  # Far future for free plan
                    )
                else:
                    return Response(
                        {"error": "No active subscription or free plan available"},
                        status=status.HTTP_404_NOT_FOUND
                    )
                
            serializer = self.get_serializer(subscription)
            return Response(serializer.data)
        except Exception as e:
            return Response(
                {"error": f"Failed to retrieve subscription: {str(e)}"},
                status=status.HTTP_500_INTERNAL_SERVER_ERROR
            )
    
    @action(detail=False, methods=['get'])
    def plans(self, request):
        """Get all available subscription plans"""
        plans = SubscriptionPlan.objects.filter(is_active=True)
        serializer = SubscriptionPlanSerializer(plans, many=True)
        return Response(serializer.data)

class UserProfileViewSet(RatelimitMixin, viewsets.ReadOnlyModelViewSet):
    serializer_class = UserProfileSerializer
    permission_classes = [permissions.IsAuthenticated]
    ratelimit_key = 'user'
    ratelimit_rate = '60/minute'
    ratelimit_method = 'GET'
    ratelimit_block = True
    
    def get_queryset(self):
        return UserProfile.objects.filter(user=self.request.user)
    
    def retrieve(self, request, *args, **kwargs):
        """Get the user's profile including subscription details"""
        try:
            profile = UserProfile.objects.get(user=request.user)
            subscription = Subscription.objects.filter(
                user=request.user, 
                is_current=True
            ).select_related('plan').first()
            
            profile_serializer = self.get_serializer(profile)
            data = profile_serializer.data
            
            # Add current subscription info
            if subscription:
                subscription_serializer = SubscriptionSerializer(subscription)
                data['subscription'] = subscription_serializer.data
            
            return Response(data)
        except UserProfile.DoesNotExist:
            return Response(
                {"error": "User profile not found"}, 
                status=status.HTTP_404_NOT_FOUND
            )
        except Exception as e:
            return Response(
                {"error": f"Failed to retrieve profile: {str(e)}"},
                status=status.HTTP_500_INTERNAL_SERVER_ERROR
            )
    
    @action(detail=False, methods=['get'])
    def leaderboard(self, request):
        """Get top credit earners (leaderboard)"""
        top_users = UserProfile.objects.order_by('-credits')[:20]
        serializer = self.get_serializer(top_users, many=True)
        return Response(serializer.data)

class CreditTransactionViewSet(RatelimitMixin, viewsets.ReadOnlyModelViewSet):
    serializer_class = CreditTransactionSerializer
    permission_classes = [permissions.IsAuthenticated]
    ratelimit_key = 'user'
    ratelimit_rate = '60/minute'
    ratelimit_method = 'GET'
    ratelimit_block = True
    
    def get_queryset(self):
        return CreditTransaction.objects.filter(user=self.request.user).order_by('-created_at')

class UseCreditsView(RatelimitMixin, APIView):
    permission_classes = [permissions.IsAuthenticated]
    ratelimit_key = 'user'
    ratelimit_rate = '20/minute'
    ratelimit_method = 'POST'
    ratelimit_block = True
    
    @transaction.atomic
    def post(self, request):
        """Use credits for a specific purpose"""
        serializer = CreditUsageSerializer(data=request.data)
        if not serializer.is_valid():
            return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
        
        amount = serializer.validated_data['amount']
        purpose = serializer.validated_data['purpose']
        
        try:
            profile = UserProfile.objects.select_for_update().get(user=request.user)
            
            # Check if user has enough credits
            if profile.credits < amount:
                return Response(
                    {"error": f"Insufficient credits. You have {profile.credits} credits, but {amount} are required."},
                    status=status.HTTP_400_BAD_REQUEST
                )
            
            # Deduct credits
            profile.credits -= amount
            profile.save()
            
            # Record transaction
            transaction = CreditTransaction.objects.create(
                user=request.user,
                amount=-amount,
                transaction_type='spend',
                description=purpose
            )
            
            # Update streak if not updated today
            today = timezone.now().date()
            if profile.last_activity_date != today:
                if profile.last_activity_date == today - datetime.timedelta(days=1):
                    # Consecutive day streak
                    profile.streak_days += 1
                else:
                    # Reset streak
                    profile.streak_days = 1
                
                profile.last_activity_date = today
                profile.save()
            
            return Response({
                "success": True,
                "message": f"Successfully used {amount} credits for {purpose}",
                "remaining_credits": profile.credits,
                "transaction_id": transaction.reference_id
            })
            
        except UserProfile.DoesNotExist:
            return Response(
                {"error": "User profile not found"}, 
                status=status.HTTP_404_NOT_FOUND
            )
        except Exception as e:
            return Response(
                {"error": f"Failed to use credits: {str(e)}"},
                status=status.HTTP_500_INTERNAL_SERVER_ERROR
            )

class ReferralView(RatelimitMixin, APIView):
    permission_classes = [permissions.IsAuthenticated]
    ratelimit_key = 'user'
    ratelimit_rate = '10/minute'
    ratelimit_method = 'POST'
    ratelimit_block = True
    
    @transaction.atomic
    def post(self, request):
        """Apply a referral code to get bonus credits"""
        serializer = ReferralSerializer(data=request.data, context={'request': request})
        if not serializer.is_valid():
            return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
        
        referral_code = serializer.validated_data['referral_code']
        
        try:
            # Get the referrer's profile
            referrer_profile = UserProfile.objects.select_for_update().get(referral_code=referral_code)
            user_profile = UserProfile.objects.select_for_update().get(user=request.user)
            
            # Check if user has already been referred
            if user_profile.referred_by is not None:
                return Response(
                    {"error": "You have already used a referral code"},
                    status=status.HTTP_400_BAD_REQUEST
                )
            
            # Set referred_by and give bonus credits
            user_profile.referred_by = referrer_profile
            
            # Give bonus to both users
            REFERRAL_BONUS = 50  # 50 credits as bonus
            referrer_profile.credits += REFERRAL_BONUS
            user_profile.credits += REFERRAL_BONUS
            
            # Save changes
            referrer_profile.save()
            user_profile.save()
            
            # Record transactions
            CreditTransaction.objects.create(
                user=referrer_profile.user,
                amount=REFERRAL_BONUS,
                transaction_type='bonus',
                description=f"Referral bonus for user {request.user.username}"
            )
            
            CreditTransaction.objects.create(
                user=request.user,
                amount=REFERRAL_BONUS,
                transaction_type='bonus',
                description=f"Bonus for using referral code {referral_code}"
            )
            
            return Response({
                "success": True,
                "message": f"Successfully applied referral code. You and the referrer each received {REFERRAL_BONUS} credits.",
                "bonus_received": REFERRAL_BONUS,
                "new_credit_balance": user_profile.credits
            })
            
        except UserProfile.DoesNotExist:
            return Response(
                {"error": "Invalid referral code or user profile not found"}, 
                status=status.HTTP_404_NOT_FOUND
            )
        except Exception as e:
            return Response(
                {"error": f"Failed to apply referral code: {str(e)}"},
                status=status.HTTP_500_INTERNAL_SERVER_ERROR
            )

In [ ]:
# subscriptions/urls.py
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from . import views

router = DefaultRouter()
router.register(r'subscriptions', views.SubscriptionViewSet, basename='subscription')
router.register(r'profile', views.UserProfileViewSet, basename='profile')
router.register(r'credits/history', views.CreditTransactionViewSet, basename='credit-history')

urlpatterns = [
    path('', include(router.urls)),
    path('credits/use/', views.UseCreditsView.as_view(), name='use-credits'),
    path('referral/apply/', views.ReferralView.as_view(), name='apply-referral'),
]

In [ ]:
# subscriptions/signals.py
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.contrib.auth import get_user_model
from .models import UserProfile, SubscriptionPlan, Subscription
from django.utils import timezone
import datetime

User = get_user_model()

@receiver(post_save, sender=User)
def create_user_profile(sender, instance, created, **kwargs):
    if created:
        # Create profile
        profile = UserProfile.objects.create(user=instance)
        
        # Assign free plan to new user
        free_plan = SubscriptionPlan.objects.filter(tier='free', is_active=True).first()
        if free_plan:
            Subscription.objects.create(
                user=instance,
                plan=free_plan,
                status='active',
                is_current=True,
                end_date=timezone.now() + datetime.timedelta(days=365*10)  # Far future for free plan
            )

In [ ]:
# subscriptions/apps.py
from django.apps import AppConfig

class SubscriptionsConfig(AppConfig):
    default_auto_field = 'django.db.models.BigAutoField'
    name = 'subscriptions'

    def ready(self):
        import subscriptions.signals

In [ ]:
# subscriptions/middleware.py
class SecurityHeadersMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        response = self.get_response(request)
        
        # Add security headers
        response['X-Content-Type-Options'] = 'nosniff'
        response['X-Frame-Options'] = 'DENY'
        response['X-XSS-Protection'] = '1; mode=block'
        response['Referrer-Policy'] = 'strict-origin-when-cross-origin'
        response['Content-Security-Policy'] = "default-src 'self'; script-src 'self' https://js.stripe.com; style-src 'self' 'unsafe-inline'; img-src 'self' data:; font-src 'self'; connect-src 'self' https://api.stripe.com;"
        response['Strict-Transport-Security'] = 'max-age=31536000; includeSubDomains'
        
        return response

In [ ]:
# settings.py additions

INSTALLED_APPS += [
    'subscriptions',
    'django_bleach',
    'django_ratelimit',
]

MIDDLEWARE += [
    'subscriptions.middleware.SecurityHeadersMiddleware',
]

# Security settings
SECURE_BROWSER_XSS_FILTER = True
SECURE_CONTENT_TYPE_NOSNIFF = True
SECURE_SSL_REDIRECT = True  # In production
SESSION_COOKIE_SECURE = True  # In production
CSRF_COOKIE_SECURE = True  # In production
CSRF_COOKIE_HTTPONLY = True
CSRF_COOKIE_SAMESITE = 'Strict'
SESSION_COOKIE_SAMESITE = 'Strict'

# Rate limiting settings
RATELIMIT_USE_CACHE = 'default'
RATELIMIT_VIEW = 'subscriptions.views.rate_limited_error'

# Bleach settings for sanitizing HTML
BLEACH_ALLOWED_TAGS = ['p', 'b', 'i', 'u', 'em', 'strong', 'a']
BLEACH_ALLOWED_ATTRIBUTES = {'a': ['href', 'title']}
BLEACH_ALLOWED_STYLES = []
BLEACH_STRIP_TAGS = True
BLEACH_STRIP_COMMENTS = True

# JWT settings
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_simplejwt.authentication.JWTAuthentication',
        'rest_framework.authentication.SessionAuthentication',
    ),
    'DEFAULT_PERMISSION_CLASSES': (
        'rest_framework.permissions.IsAuthenticated',
    ),
    'DEFAULT_THROTTLE_CLASSES': [
        'rest_framework.throttling.AnonRateThrottle',
        'rest_framework.throttling.UserRateThrottle'
    ],
    'DEFAULT_THROTTLE_RATES': {
        'anon': '20/minute',
        'user': '60/minute'
    },
    'DEFAULT_RENDERER_CLASSES': (
        'rest_framework.renderers.JSONRenderer',
    ),
    'EXCEPTION_HANDLER': 'subscriptions.utils.custom_exception_handler',
}

# Add to main urls.py
# urls.py
urlpatterns += [
    path('api/subscriptions/', include('subscriptions.urls')),
]

In [ ]:
# settings.py additions

INSTALLED_APPS += [
    'subscriptions',
    'django_bleach',
    'django_ratelimit',
]

MIDDLEWARE += [
    'subscriptions.middleware.SecurityHeadersMiddleware',
]

# Security settings
SECURE_BROWSER_XSS_FILTER = True
SECURE_CONTENT_TYPE_NOSNIFF = True
SECURE_SSL_REDIRECT = True  # In production
SESSION_COOKIE_SECURE = True  # In production
CSRF_COOKIE_SECURE = True  # In production
CSRF_COOKIE_HTTPONLY = True
CSRF_COOKIE_SAMESITE = 'Strict'
SESSION_COOKIE_SAMESITE = 'Strict'

# Rate limiting settings
RATELIMIT_USE_CACHE = 'default'
RATELIMIT_VIEW = 'subscriptions.views.rate_limited_error'

# Bleach settings for sanitizing HTML
BLEACH_ALLOWED_TAGS = ['p', 'b', 'i', 'u', 'em', 'strong', 'a']
BLEACH_ALLOWED_ATTRIBUTES = {'a': ['href', 'title']}
BLEACH_ALLOWED_STYLES = []
BLEACH_STRIP_TAGS = True
BLEACH_STRIP_COMMENTS = True

# JWT settings
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_simplejwt.authentication.JWTAuthentication',
        'rest_framework.authentication.SessionAuthentication',
    ),
    'DEFAULT_PERMISSION_CLASSES': (
        'rest_framework.permissions.IsAuthenticated',
    ),
    'DEFAULT_THROTTLE_CLASSES': [
        'rest_framework.throttling.AnonRateThrottle',
        'rest_framework.throttling.UserRateThrottle'
    ],
    'DEFAULT_THROTTLE_RATES': {
        'anon': '20/minute',
        'user': '60/minute'
    },
    'DEFAULT_RENDERER_CLASSES': (
        'rest_framework.renderers.JSONRenderer',
    ),
    'EXCEPTION_HANDLER': 'subscriptions.utils.custom_exception_handler',
}

# Add to main urls.py
# urls.py
urlpatterns += [
    path('api/subscriptions/', include('subscriptions.urls')),
]

In [ ]:
python manage.py makemigrations subscriptions
python manage.py migrate

In [ ]:
# subscriptions/management/commands/setup_subscription_plans.py
from django.core.management.base import BaseCommand
from subscriptions.models import SubscriptionPlan

class Command(BaseCommand):
    help = 'Sets up the initial subscription plans'

    def handle(self, *args, **options):
        # Personal plans
        personal_plans = [
            {
                'name': 'Personal Free',
                'description': 'Basic access with limited features',
                'price': 0.00,
                'type': 'personal',
                'tier': 'free',
                'credits_per_month': 50,
                'features': {
                    'basic_access': True,
                    'premium_content': False,
                    'api_access': False,
                    'priority_support': False,
                }
            },
            {
                'name': 'Personal Pro',
                'description': 'Enhanced access with more credits and features',
                'price': 9.99,
                'type': 'personal',
                'tier': 'pro',
                'credits_per_month': 200,
                'features': {
                    'basic_access': True,
                    'premium_content': True,
                    'api_access': False,
                    'priority_support': False,
                }
            },
            {
                'name': 'Personal Max',
                'description': 'Full access to all personal features and priority support',
                'price': 19.99,
                'type': 'personal',
                'tier': 'max',
                'credits_per_month': 500,
                'features': {
                    'basic_access': True,
                    'premium_content': True,
                    'api_access': True,
                    'priority_support': True,
                }
            },
        ]
        
        # Business plans
        business_plans = [
            {
                'name': 'Business Team',
                'description': 'Perfect for small teams with shared access',
                'price': 49.99,
                'type': 'business',
                'tier': 'team',
                'credits_per_month': 1000,
                'max_users': 5,
                'features': {
                    'basic_access': True,
                    'premium_content': True,
                    'api_access': True,
                    'priority_support': True,
                    'team_management': True,
                }
            },
            {
                'name': 'Business Enterprise',
                'description': 'Full enterprise solution with unlimited users',
                'price': 199.99,
                'type': 'business',
                'tier': 'enterprise',
                'credits_per_month': 5000,
                'max_users': 100,
                'features': {
                    'basic_access': True,
                    'premium_content': True,
                    'api_access': True,
                    'priority_support': True,
                    'team_management': True,
                    'custom_integration': True,
                    'dedicated_account_manager': True,
                }
            },
        ]
        
        plans_created = 0
        
        # Create personal plans
        for plan_data in personal_plans:
            plan, created = SubscriptionPlan.objects.update_or_create(
                type=plan_data['type'],
                tier=plan_data['tier'],
                defaults=plan_data
            )
            if created:
                plans_created += 1
                self.stdout.write(f"Created plan: {plan.name}")
            else:
                self.stdout.write(f"Updated plan: {plan.name}")
        
        # Create business plans
        for plan_data in business_plans:
            plan, created = SubscriptionPlan.objects.update_or_create(
                type=plan_data['type'],
                tier=plan_data['tier'],
                defaults=plan_data
            )
            if created:
                plans_created += 1
                self.stdout.write(f"Created plan: {plan.name}")
            else:
                self.stdout.write(f"Updated plan: {plan.name}")
        
        self.stdout.write(self.style.SUCCESS(f'Successfully set up {plans_created} subscription plans'))

In [ ]:
// src/lib/api/subscription.ts

import axios from 'axios';
import { getSession } from 'next-auth/react';

// Types
export interface SubscriptionPlan {
  id: number;
  name: string;
  description: string;
  price: number;
  type: 'personal' | 'business';
  tier: 'free' | 'pro' | 'max' | 'team' | 'enterprise';
  credits_per_month: number;
  max_users: number;
  features: Record<string, boolean>;
}

export interface Subscription {
  id: number;
  plan: number;
  plan_details: SubscriptionPlan;
  status: 'active' | 'canceled' | 'expired' | 'trial';
  start_date: string;
  end_date: string | null;
}

export interface UserProfile {
  id: number;
  email: string;
  username: string;
  credits: number;
  referral_code: string;
  streak_days: number;
  subscription?: Subscription;
}

export interface CreditTransaction {
  id: number;
  amount: number;
  transaction_type: 'earn' | 'spend' | 'bonus' | 'refund' | 'subscription';
  description: string;
  reference_id: string;
  created_at: string;
}

export interface UseCreditsRequest {
  amount: number;
  purpose: string;
}

export interface UseCreditsResponse {
  success: boolean;
  message: string;
  remaining_credits: number;
  transaction_id: string;
}

export interface ReferralRequest {
  referral_code: string;
}

export interface ReferralResponse {
  success: boolean;
  message: string;
  bonus_received: number;
  new_credit_balance: number;
}

// API URL - this should come from environment variables
const API_URL = process.env.NEXT_PUBLIC_API_URL || 'https://iamdreamer.ecuenex.com/api';

// Helper function to get auth headers
const getAuthHeaders = async () => {
  const session = await getSession();
  
  if (!session?.accessToken) {
    throw new Error('Authentication required');
  }
  
  return {
    Authorization: `Bearer ${session.accessToken}`,
    'Content-Type': 'application/json',
  };
};

// API client for subscription-related endpoints
export const subscriptionApi = {
  // Get current user subscription
  getCurrentSubscription: async (): Promise<Subscription> => {
    try {
      const headers = await getAuthHeaders();
      const response = await axios.get(`${API_URL}/subscriptions/current/`, { headers });
      return response.data;
    } catch (error) {
      console.error('Error fetching subscription:', error);
      throw error;
    }
  },
  
  // Get all available subscription plans
  getSubscriptionPlans: async (): Promise<SubscriptionPlan[]> => {
    try {
      const headers = await getAuthHeaders();
      const response = await axios.get(`${API_URL}/subscriptions/plans/`, { headers });
      return response.data;
    } catch (error) {
      console.error('Error fetching subscription plans:', error);
      throw error;
    }
  },
  
  // Get user profile with subscription details
  getUserProfile: async (): Promise<UserProfile> => {
    try {
      const headers = await getAuthHeaders();
      const response = await axios.get(`${API_URL}/profile/current/`, { headers });
      return response.data;
    } catch (error) {
      console.error('Error fetching user profile:', error);
      throw error;
    }
  },
  
  // Get credit transaction history
  getCreditHistory: async (): Promise<CreditTransaction[]> => {
    try {
      const headers = await getAuthHeaders();
      const response = await axios.get(`${API_URL}/credits/history/`, { headers });
      return response.data;
    } catch (error) {
      console.error('Error fetching credit history:', error);
      throw error;
    }
  },
  
  // Get leaderboard (top credit earners)
  getLeaderboard: async (): Promise<UserProfile[]> => {
    try {
      const headers = await getAuthHeaders();
      const response = await axios.get(`${API_URL}/profile/leaderboard/`, { headers });
      return response.data;
    } catch (error) {
      console.error('Error fetching leaderboard:', error);
      throw error;
    }
  },
  
  // Use credits for a specific purpose
  useCredits: async (data: UseCreditsRequest): Promise<UseCreditsResponse> => {
    try {
      const headers = await getAuthHeaders();
      const response = await axios.post(`${API_URL}/credits/use/`, data, { headers });
      return response.data;
    } catch (error) {
      console.error('Error using credits:', error);
      throw error;
    }
  },
  
  // Apply a referral code
  applyReferralCode: async (data: ReferralRequest): Promise<ReferralResponse> => {
    try {
      const headers = await getAuthHeaders();
      const response = await axios.post(`${API_URL}/referral/apply/`, data, { headers });
      return response.data;
    } catch (error) {
      console.error('Error applying referral code:', error);
      throw error;
    }
  }
};

In [ ]:
// src/components/subscription/SubscriptionDetails.tsx
import React, { useEffect, useState } from 'react';
import { useRouter } from 'next/router';
import { 
  Subscription, 
  UserProfile, 
  subscriptionApi 
} from '@/lib/api/subscription';
import { useSession } from 'next-auth/react';
import { Alert, AlertDescription } from '@/components/ui/alert';
import { Card, CardContent, CardDescription, CardFooter, CardHeader, CardTitle } from '@/components/ui/card';
import { Badge } from '@/components/ui/badge';

export default function SubscriptionDetails() {
  const { data: session, status } = useSession();
  const router = useRouter();
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState<string | null>(null);
  const [profile, setProfile] = useState<UserProfile | null>(null);
  
  useEffect(() => {
    // Redirect if not authenticated
    if (status === 'unauthenticated') {
      router.push('/login');
      return;
    }
    
    if (status === 'authenticated') {
      fetchUserProfile();
    }
  }, [status, router]);
  
  const fetchUserProfile = async () => {
    try {
      setLoading(true);
      const profileData = await subscriptionApi.getUserProfile();
      setProfile(profileData);
      setError(null);
    } catch (err: any) {
      setError(err.response?.data?.error || 'Failed to load subscription details');
      console.error('Error fetching profile:', err);
    } finally {
      setLoading(false);
    }
  };
  
  if (loading) {
    return <div className="p-8 flex justify-center">Loading subscription details...</div>;
  }
  
  if (error) {
    return (
      <Alert variant="destructive" className="mb-6">
        <AlertDescription>{error}</AlertDescription>
      </Alert>
    );
  }
  
  if (!profile) {
    return (
      <Alert className="mb-6">
        <AlertDescription>No subscription information available</AlertDescription>
      </Alert>
    );
  }
  
  const subscription = profile.subscription;
  
  return (
    <div className="max-w-4xl mx-auto p-4">
      <Card className="mb-8">
        <CardHeader>
          <div className="flex justify-between items-center">
            <CardTitle>Subscription Details</CardTitle>
            {subscription && (
              <Badge 
                className={
                  subscription.status === 'active' ? 'bg-green-500' : 
                  subscription.status === 'trial' ? 'bg-blue-500' : 
                  'bg-red-500'
                }
              >
                {subscription.status.toUpperCase()}
              </Badge>
            )}
          </div>
          <CardDescription>
            Your current subscription plan and credits
          </CardDescription>
        </CardHeader>
        
        <CardContent>
          {subscription ? (
            <div className="space-y-4">
              <div>
                <h3 className="font-medium text-lg">{subscription.plan_details.name}</h3>
                <p className="text-muted-foreground">{subscription.plan_details.description}</p>
              </div>
              
              <div className="grid grid-cols-1 md:grid-cols-2 gap-4">
                <div>
                  <p className="text-sm font-medium">Price</p>
                  <p className="text-2xl font-bold">
                    ${subscription.plan_details.price.toFixed(2)}
                    <span className="text-sm font-normal">/month</span>
                  </p>
                </div>
                
                <div>
                  <p className="text-sm font-medium">Monthly Credits</p>
                  <p className="text-2xl font-bold">
                    {subscription.plan_details.credits_per_month}
                  </p>
                </div>
              </div>
              
              <div>
                <p className="text-sm font-medium mb-2">Plan Features</p>
                <ul className="space-y-2">
                  {Object.entries(subscription.plan_details.features).map(([key, value]) => (
                    <li key={key} className="flex items-center">
                      {value ? (
                        <svg className="w-5 h-5 mr-2 text-green-500" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                          <path strokeLinecap="round" strokeLinejoin="round" strokeWidth={2} d="M5 13l4 4L19 7" />
                        </svg>
                      ) : (
                        <svg className="w-5 h-5 mr-2 text-red-500" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                          <path strokeLinecap="round" strokeLinejoin="round" strokeWidth={2} d="M6 18L18 6M6 6l12 12" />
                        </svg>
                      )}
                      {key.split('_').map(word => word.charAt(0).toUpperCase() + word.slice(1)).join(' ')}
                    </li>
                  ))}
                </ul>
              </div>
              
              {subscription.end_date && (
                <div>
                  <p className="text-sm font-medium">Valid Until</p>
                  <p>{new Date(subscription.end_date).toLocaleDateString()}</p>
                </div>
              )}
            </div>
          ) : (
            <p>No active subscription found. Choose a subscription plan to get started.</p>
          )}
        </CardContent>
        
        <CardFooter className="flex justify-between">
          <div>
            <span className="font-medium">Current Credits:</span> {profile.credits}
          </div>
          {subscription && subscription.plan_details.tier !== 'free' && (
            <button 
              className="px-4 py-2 bg-red-600 text-white rounded hover:bg-red-700"
              onClick={() => {
                // Handle cancel subscription
                // This would ideally open a confirmation modal
                console.log('Cancel subscription clicked');
              }}
            >
              Cancel Subscription
            </button>
          )}
        </CardFooter>
      </Card>
    </div>
  );
}

In [ ]:
// src/components/subscription/CreditHistory.tsx
import React, { useEffect, useState } from 'react';
import { useSession } from 'next-auth/react';
import { useRouter } from 'next/router';
import { 
  CreditTransaction, 
  subscriptionApi 
} from '@/lib/api/subscription';
import { Alert, AlertDescription } from '@/components/ui/alert';
import { Card, CardContent, CardDescription, CardHeader, CardTitle } from '@/components/ui/card';
import { Table, TableBody, TableCell, TableHead, TableHeader, TableRow } from '@/components/ui/table';
import { Badge } from '@/components/ui/badge';

export default function CreditHistory() {
  const { data: session, status } = useSession();
  const router = useRouter();
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState<string | null>(null);
  const [transactions, setTransactions] = useState<CreditTransaction[]>([]);
  
  useEffect(() => {
    // Redirect if not authenticated
    if (status === 'unauthenticated') {
      router.push('/login');
      return;
    }
    
    if (status === 'authenticated') {
      fetchCreditHistory();
    }
  }, [status, router]);
  
  const fetchCreditHistory = async () => {
    try {
      setLoading(true);
      const history = await subscriptionApi.getCreditHistory();
      setTransactions(history);
      setError(null);
    } catch (err: any) {
      setError(err.response?.data?.error || 'Failed to load credit history');
      console.error('Error fetching credit history:', err);
    } finally {
      setLoading(false);
    }
  };
  
  const getTransactionTypeColor = (type: string) => {
    switch (type) {
      case 'earn':
        return 'bg-green-500';
      case 'spend':
        return 'bg-red-500';
      case 'bonus':
        return 'bg-purple-500';
      case 'refund':
        return 'bg-blue-500';
      case 'subscription':
        return 'bg-yellow-500';
      default:
        return 'bg-gray-500';
    }
  };
  
  if (loading) {
    return <div className="p-8 flex justify-center">Loading credit history...</div>;
  }
  
  if (error) {
    return (
      <Alert variant="destructive" className="mb-6">
        <AlertDescription>{error}</AlertDescription>
      </Alert>
    );
  }
  
  return (
    <div className="max-w-4xl mx-auto p-4">
      <Card>
        <CardHeader>
          <CardTitle>Credit History</CardTitle>
          <CardDescription>
            Your credit transactions and activity
          </CardDescription>
        </CardHeader>
        
        <CardContent>
          {transactions.length > 0 ? (
            <Table>
              <TableHeader>
                <TableRow>
                  <TableHead>Date</TableHead>
                  <TableHead>Type</TableHead>
                  <TableHead>Description</TableHead>
                  <TableHead className="text-right">Amount</TableHead>
                </TableRow>
              </TableHeader>
              <TableBody>
                {transactions.map((transaction) => (
                  <TableRow key={transaction.id}>
                    <TableCell className="font-medium">
                      {new Date(transaction.created_at).toLocaleDateString()}
                    </TableCell>
                    <TableCell>
                      <Badge className={getTransactionTypeColor(transaction.transaction_type)}>
                        {transaction.transaction_type.toUpperCase()}
                      </Badge>
                    </TableCell>
                    <TableCell>{transaction.description}</TableCell>
                    <TableCell className={`text-right ${transaction.amount > 0 ? 'text-green-600' : 'text-red-600'}`}>
                      {transaction.amount > 0 ? '+' : ''}{transaction.amount}
                    </TableCell>
                  </TableRow>
                ))}
              </TableBody>
            </Table>
          ) : (
            <p className="text-center py-8">No credit transactions found.</p>
          )}
        </CardContent>
      </Card>
    </div>
  );
}

In [ ]:
// src/components/subscription/Leaderboard.tsx
import React, { useEffect, useState } from 'react';
import { UserProfile, subscriptionApi } from '@/lib/api/subscription';
import { Alert, AlertDescription } from '@/components/ui/alert';
import { Card, CardContent, CardDescription, CardHeader, CardTitle } from '@/components/ui/card';
import { Table, TableBody, TableCell, TableHead, TableHeader, TableRow } from '@/components/ui/table';

export default function Leaderboard() {
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState<string | null>(null);
  const [leaders, setLeaders] = useState<UserProfile[]>([]);
  
  useEffect(() => {
    fetchLeaderboard();
  }, []);
  
  const fetchLeaderboard = async () => {
    try {
      setLoading(true);
      const leaderboardData = await subscriptionApi.getLeaderboard();
      setLeaders(leaderboardData);
      setError(null);
    } catch (err: any) {
      setError(err.response?.data?.error || 'Failed to load leaderboard');
      console.error('Error fetching leaderboard:', err);
    } finally {
      setLoading(false);
    }
  };
  
  if (loading) {
    return <div className="p-8 flex justify-center">Loading leaderboard...</div>;
  }
  
  if (error) {
    return (
      <Alert variant="destructive" className="mb-6">
        <AlertDescription>{error}</AlertDescription>
      </Alert>
    );
  }
  
  return (
    <div className="max-w-4xl mx-auto p-4">
      <Card>
        <CardHeader>
          <CardTitle>Credit Leaders</CardTitle>
          <CardDescription>
            Top users with the most credits
          </CardDescription>
        </CardHeader>
        
        <CardContent>
          {leaders.length > 0 ? (
            <Table>
              <TableHeader>
                <TableRow>
                  <TableHead>Rank</TableHead>
                  <TableHead>Username</TableHead>
                  <TableHead className="text-right">Credits</TableHead>
                  <TableHead className="text-right">Consecutive Days</TableHead>
                </TableRow>
              </TableHeader>
              <TableBody>
                {leaders.map((user, index) => (
                  <TableRow key={user.id} className={index < 3 ? "bg-amber-50" : ""}>
                    <TableCell className="font-medium">
                      {index === 0 ? (
                        <span className="inline-flex items-center justify-center w-6 h-6 bg-yellow-500 text-white rounded-full">1</span>
                      ) : index === 1 ? (
                        <span className="inline-flex items-center justify-center w-6 h-6 bg-gray-400 text-white rounded-full">2</span>
                      ) : index === 2 ? (
                        <span className="inline-flex items-center justify-center w-6 h-6 bg-amber-700 text-white rounded-full">3</span>
                      ) : (
                        <span>{index + 1}</span>
                      )}
                    </TableCell>
                    <TableCell>
                      {user.username}
                      {index < 3 && (
                        <span className="ml-2">
                          {index === 0 ? '🥇' : index === 1 ? '🥈' : '🥉'}
                        </span>
                      )}
                    </TableCell>
                    <TableCell className="text-right font-bold">
                      {user.credits}
                    </TableCell>
                    <TableCell className="text-right">
                      {user.streak_days} {user.streak_days > 0 && '🔥'}
                    </TableCell>
                  </TableRow>
                ))}
              </TableBody>
            </Table>
          ) : (
            <p className="text-center py-8">No leaders found.</p>
          )}
        </CardContent>
      </Card>
    </div>
  );
}

In [ ]:
// src/components/subscription/ReferralSystem.tsx
import React, { useEffect, useState } from 'react';
import { useSession } from 'next-auth/react';
import { useRouter } from 'next/router';
import { 
  UserProfile, 
  ReferralRequest,
  subscriptionApi 
} from '@/lib/api/subscription';
import { Alert, AlertDescription, AlertTitle } from '@/components/ui/alert';
import { Card, CardContent, CardDescription, CardHeader, CardTitle } from '@/components/ui/card';
import { Button } from '@/components/ui/button';
import { Input } from '@/components/ui/input';
import { Label } from '@/components/ui/label';
import { Copy } from 'lucide-react';

export default function ReferralSystem() {
  const { data: session, status } = useSession();
  const router = useRouter();
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState<string | null>(null);
  const [success, setSuccess] = useState<string | null>(null);
  const [profile, setProfile] = useState<UserProfile | null>(null);
  const [referralCode, setReferralCode] = useState<string>('');
  const [copied, setCopied] = useState(false);
  
  useEffect(() => {
    // Redirect if not authenticated
    if (status === 'unauthenticated') {
      router.push('/login');
      return;
    }
    
    if (status === 'authenticated') {
      fetchUserProfile();
    }
  }, [status, router]);
  
  const fetchUserProfile = async () => {
    try {
      setLoading(true);
      const profileData = await subscriptionApi.getUserProfile();
      setProfile(profileData);
      setError(null);
    } catch (err: any) {
      setError(err.response?.data?.error || 'Failed to load profile');
      console.error('Error fetching profile:', err);
    } finally {
      setLoading(false);
    }
  };
  
  const handleCopyReferralCode = () => {
    if (profile?.referral_code) {
      navigator.clipboard.writeText(profile.referral_code);
      setCopied(true);
      setTimeout(() => setCopied(false), 2000);
    }
  };
  
  const handleSubmitReferralCode = async (e: React.FormEvent) => {
    e.preventDefault();
    
    if (!referralCode.trim()) {
      setError('Please enter a valid referral code');
      return;
    }
    
    try {
      setLoading(true);
      setError(null);
      setSuccess(null);
      
      const request: ReferralRequest = { referral_code: referralCode.trim() };
      const response = await subscriptionApi.applyReferralCode(request);
      
      setSuccess(response.message);
      setReferralCode('');
      
      // Refresh user profile to show updated credits
      await fetchUserProfile();
    } catch (err: any) {
      setError(err.response?.data?.error || 'Failed to apply referral code');
      console.error('Error applying referral code:', err);
    } finally {
      setLoading(false);
    }
  };
  
  if (loading && !profile) {
    return <div className="p-8 flex justify-center">Loading referral system...</div>;
  }
  
  return (
    <div className="max-w-4xl mx-auto p-4">
      <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
        {/* Your Referral Code */}
        <Card>
          <CardHeader>
            <CardTitle>Your Referral Code</CardTitle>
            <CardDescription>
              Share your referral code with friends to earn bonus credits
            </CardDescription>
          </CardHeader>
          
          <CardContent>
            {profile ? (
              <div className="space-y-4">
                <div className="flex items-center space-x-2">
                  <div className="relative flex-1">
                    <Input 
                      value={profile.referral_code} 
                      readOnly 
                      className="pr-10 font-mono text-center text-lg"
                    />
                    <button
                      className="absolute right-2 top-1/2 transform -translate-y-1/2"
                      onClick={handleCopyReferralCode}
                      aria-label="Copy referral code"
                    >
                      <Copy size={18} />
                    </button>
                  </div>
                </div>
                
                {copied && (
                  <p className="text-sm text-green-600 text-center">Copied to clipboard!</p>
                )}
                
                <div className="bg-blue-50 p-4 rounded-md">
                  <p className="text-sm">
                    <strong>How it works:</strong> You get 50 bonus credits for each friend who signs up using your code. 
                    They also get 50 bonus credits when they enter your code!
                  </p>
                </div>
              </div>
            ) : (
              <p>Loading your referral code...</p>
            )}
          </CardContent>
        </Card>
        
        {/* Apply Referral Code */}
        <Card>
          <CardHeader>
            <CardTitle>Apply a Referral Code</CardTitle>
            <CardDescription>
              Enter a friend's referral code to get bonus credits
            </CardDescription>
          </CardHeader>
          
          <CardContent>
            <form onSubmit={handleSubmitReferralCode} className="space-y-4">
              <div className="space-y-2">
                <Label htmlFor="referralCode">Friend's Referral Code</Label>
                <Input
                  id="referralCode"
                  placeholder="Enter code (e.g., AB12CD34)"
                  value={referralCode}
                  onChange={(e) => setReferralCode(e.target.value.toUpperCase())}
                  maxLength={8}
                  className="font-mono"
                />
              </div>
              
              {error && (
                <Alert variant="destructive">
                  <AlertDescription>{error}</AlertDescription>
                </Alert>
              )}
              
              {success && (
                <Alert className="bg-green-50 border-green-200">
                  <AlertTitle>Success!</AlertTitle>
                  <AlertDescription>{success}</AlertDescription>
                </Alert>
              )}
              
              <Button 
                type="submit" 
                className="w-full" 
                disabled={loading || !referralCode.trim()}
              >
                {loading ? 'Processing...' : 'Apply Code'}
              </Button>
            </form>
          </CardContent>
        </Card>
      </div>
    </div>
  );
}